In [ ]:
# 回译
from pygtrans import Translate

def backTran(wordss):
    client = Translate()
    text1 = client.translate(wordss)

    text2 = client.translate(text1.translatedText, target='en')
    return text2.translatedText

In [ ]:
#同义词替换
from nltk.corpus import stopwords#引入停用词，因为对停用词进行数据增强相当于没有增强
from nltk.corpus import wordnet as wn#引入同义词
import random
stop_words=stopwords.words('english')
for w in ['!',',','.','?','-s','-ly','</s>','s']:
    stop_words.add(w)
   
#这里传入的words是一个列表,
#eg:"hello world".split(" ") or ["hello","world"]
def synonym_replacement(words):
    new_words = words.copy()
    n=len(words)/4
    random_word_list = list(set([word for word in words if word not in stop_words]))     
    random.shuffle(random_word_list)
    num_replaced = 0  
    for random_word in random_word_list:          
        synonyms = get_synonyms(random_word)
        if len(synonyms) >= 1:
            synonym = random.choice(synonyms)   
            new_words = [synonym if word == random_word else word for word in new_words]   
            num_replaced += 1
        if num_replaced >= n: 
            break

    sentence = ' '.join(new_words)
    new_words = sentence.split(' ')

    return " ".join(new_words)

#获取同义词
def get_synonyms(word):
    nearbyWordSet=wn.synsets(word)
    return nearbyWordSet[0].lemma_names()

In [5]:
########################################################################
# 随机插入
# 随机在语句中插入n个词
########################################################################
#这里传入的words是一个列表,
#eg:"hello world".split(" ") or ["hello","world"]
def random_insertion(words):
    new_words = words.copy()
    n=len(new_words)/n
    for _ in range(n):
        add_word(new_words)
    return " ".join(new_words)
#插入单词，这里插入随机挑选的单词的同义词
def add_word(new_words):
    synonyms = []
    counter = 0    
    while len(synonyms) < 1:
        random_word = new_words[random.randint(0, len(new_words)-1)]
        synonyms = get_synonyms(random_word)
        counter += 1
        if counter >= 10:
            return
    random_synonym = random.choice(synonyms)
    random_idx = random.randint(0, len(new_words)-1)
    new_words.insert(random_idx, random_synonym)


None
peope
likee


In [ ]:
########################################################################
# 随机交换
# 随机交换几次
########################################################################
#这里传入的words是一个列表,
#eg:"hello world".split(" ") or ["hello","world"]
def random_swap(words):
    new_words = words.copy()
    n=len(new_words)/4
    for _ in range(n):
        new_words = swap_word(new_words)
    return " ".join(new_words)

def swap_word(new_words):
    random_idx_1 = random.randint(0, len(new_words)-1)
    random_idx_2 = random_idx_1
    counter = 0
    while random_idx_2 == random_idx_1:
        random_idx_2 = random.randint(0, len(new_words)-1)
        counter += 1
        if counter > 3:
            return new_words
    new_words[random_idx_1], new_words[random_idx_2] = new_words[random_idx_2], new_words[random_idx_1] 
    return new_words


In [ ]:
########################################################################
# 随机删除
# 以概率p删除语句中的词
########################################################################
#这里传入的words是一个列表,
#eg:"hello world".split(" ") or ["hello","world"]
def random_deletion(words):

    if len(words) == 1:
        return words

    new_words = []
    for word in words:
        r = random.uniform(0, 1)
        if r > 0.25:
            new_words.append(word)

    if len(new_words) == 0:
        rand_int = random.randint(0, len(words)-1)
        return [words[rand_int]]

    return " ".join(new_words)

In [7]:
def getEda(df,func):
    title=df.title.tolist()
    abstract=df.abstract.tolist()
    categories=df.categories.tolist()
    len1=len(title)
    exTitle=[]
    exabs=[]
    excat=[]
    for i in range(len1):
        lentitle=len(title[i])
        lenabstrct=len(abstract[i])
        txtitle=func(title[i].split(" "))#这里可以换随机插入，同义词替换，随机删除，随机交换的函数
        txabs=func(abstract[i].split(" "))#但是传入的时候用split()函数分割成一个list
        if i%1000==0:
            print(i)
        exTitle.append(txtitle)
        exabs.append(txabs)
        excat.append(categories[i])  
    return exTitle,exabs,excat

In [36]:
def getBackTran(df,func):
    title=df.title.tolist()
    abstract=df.abstract.tolist()
    categories=df.categories.tolist()
    len1=len(title)
    exTitle=[]
    exabs=[]
    excat=[]
    for i in range(len1):
        lentitle=len(title[i])
        lenabstrct=len(abstract[i])
        txtitle=func(title[i])#这里可以换随机插入，同义词替换，随机删除，随机交换的函数
        txabs=func(abstract[i])#但是传入的时候用split()函数分割成一个list
        if i%1000==0:
            print(i)
        exTitle.append(txtitle)
        exabs.append(txabs)
        excat.append(categories[i])  
    return exTitle,exabs,excat

'Effect of different substrates on Compact stacked square Microstrip\n  Antenna'

In [37]:
#读取数据
import pandas as pd
import numpy as np
df=pd.read_csv("./train.csv".sep='\t')

In [23]:
edaData=df.sample(frac=1.0)

In [34]:
#划分数据
eda1=edaData[0:12500]

eda2=edaData[12500:25000]

eda3=edaData[25000:37500]

eda4=edaData[37500:50000]


In [38]:
#分别使用不同的方法来扩增数据
extitle,exabstract,txtype=getEda(eda1,synonym_replacement)
edaData['title'].extend(extitle)
edaData['abstract'].extend(extitle)
edaData['categories'].extend(txtype)
extitle,exabstract,txtype=getEda(eda2,random_insertion)
edaData['title'].extend(extitle)
edaData['abstract'].extend(extitle)
edaData['categories'].extend(txtype)
extitle,exabstract,txtype=getEda(eda3,random_swap)
edaData['title'].extend(extitle)
edaData['abstract'].extend(extitle)
edaData['categories'].extend(txtype)
extitle,exabstract,txtype=getEda(eda4,random_deletion)
edaData['title'].extend(extitle)
edaData['abstract'].extend(extitle)
edaData['categories'].extend(txtype)

In [ ]:
#封装数据
edaData.to_csv("./edaData.csv")

In [ ]:
#回译
df=pd.read_csv("./trian.csv",sep='\t')
extitle,exabstract,extype=getBackTran(df,backTran)
df['title'].extend(extitle)
df['abstract'].extend(extitle)
df['categories'].extend(extype)